In [1]:
import logging, sys, yaml, random, os, numpy as np
from fmcml.data import load_data, load_splitter
from echo.src.base_objective import BaseObjective
from sklearn.preprocessing import StandardScaler, QuantileTransformer
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score, mean_absolute_percentage_error
from collections import defaultdict
import xgboost as xgb
import pandas as pd
import tqdm, torch, shutil, os, joblib

2022-12-20 11:24:08.804202: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F AVX512_VNNI FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2022-12-20 11:24:09.152137: I tensorflow/core/util/util.cc:169] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2022-12-20 11:24:09.197356: E tensorflow/stream_executor/cuda/cuda_blas.cc:2981] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2022-12-20 11:24:11.337147: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; 

In [5]:
config = "../results/live/echo_xgb/best.yml"
with open(config) as cf:
    conf = yaml.load(cf, Loader=yaml.FullLoader)

In [14]:
seed = conf["seed"]

save_loc = conf["save_loc"]
data_loc = conf["data_loc"]
#input_vars = conf["input_vars"]
total_input_vars = []
if conf["use_nwm"]:
    total_input_vars += conf["nwm_vars"]
if conf["use_sat"]:
    total_input_vars += conf["sat_vars"]
if conf["use_static"]:
    total_input_vars += conf["static_vars"]
if conf["use_hrrr"]:
    total_input_vars += conf["hrrr_vars"]
if conf["use_lst"]:
    total_input_vars += conf["lst_vars"]

if len(total_input_vars) == 0:
    if trial is not False:
        raise optuna.TrialPruned()
    else:
        raise OSError(
            "ECHO suggested no input columns, sigaling to prune this trial")

#static_vars = conf["static_vars"]
output_vars = conf["output_vars"]
verbose = conf["verbose"]
#total_input_vars = input_vars + static_vars
splitter = conf["split_type"]

# model config
n_splits = conf["model"]["n_splits"]
objective = conf["model"]["objective"]
learning_rate = conf["model"]["learning_rate"]
n_estimators = conf["model"]["n_estimators"]
max_depth = conf["model"]["max_depth"]
n_jobs = conf["model"]["n_jobs"]
colsample_bytree = conf["model"]["colsample_bytree"]
gamma = conf["model"]["gamma"]
learning_rate = conf["model"]["learning_rate"]
max_depth = conf["model"]["max_depth"]
subsample = conf["model"]["subsample"]
metric = conf["model"]["metric"]

# Load the data frame with filtered columns 
if "csv" in data_loc:
    df = pd.read_csv(data_loc)
    filter_nans = []
    if isinstance(total_input_vars, list):
        filter_nans += total_input_vars
    if isinstance(output_vars, list):
        filter_nans += output_vars
    if len(filter_nans):
        if verbose: 
            logging.info(f"Starting df size: {df.shape}")
            logging.info(f"Filtering NaNs from columns: {filter_nans}")
        filter_condition = df[filter_nans].isna().sum(axis=1).astype(bool)
        df = df[~filter_condition].copy()
        logging.info(f"Training df size after removing NaNs: {df.shape}")
else:
    df = load_data(
        data_loc,
        verbose = verbose,
        filter_input_vars = total_input_vars, 
        filter_output_vars = output_vars
    )

In [ ]:
def vegetatation_indices(df):
    # Normalized difference vegetation index
    df["ndvi"] = (df["375m Surface Reflectance Band I2_medium"] - df["375m Surface Reflectance Band I1_medium"]) 
    df["ndvi"] /= (df["375m Surface Reflectance Band I2_medium"] + df["375m Surface Reflectance Band I1_medium"]) 
    # Normalized difference water index
    df["ndwi_1"] = (df["375m Surface Reflectance Band I2_medium"] - df["750m Surface Reflectance Band M8_medium"])
    df["ndwi_1"] /= (df["375m Surface Reflectance Band I2_medium"] + df["750m Surface Reflectance Band M8_medium"])
    df["ndwi_2"] = (df["750m Surface Reflectance Band M7_medium"] - df["750m Surface Reflectance Band M8_medium"]) 
    df["ndwi_2"] /= (df["750m Surface Reflectance Band M7_medium"] + df["750m Surface Reflectance Band M8_medium"])
    # Normalized difference Infrared index
    #df["ndii_1"] = (df["750m Surface Reflectance Band M6_medium"] - df["750m Surface Reflectance Band M10_medium"])
    #df["ndii_1"] /= (df["750m Surface Reflectance Band M6_medium"] + df["750m Surface Reflectance Band M10_medium"])
    df["ndii_2"] = (df["750m Surface Reflectance Band M7_medium"] - df["750m Surface Reflectance Band M10_medium"])
    df["ndii_2"] /= (df["750m Surface Reflectance Band M7_medium"] + df["750m Surface Reflectance Band M10_medium"])
    # Enhanced Vegetation Index
    df["evi"] = 2.5 * (df['375m Surface Reflectance Band I2_medium'] - df['375m Surface Reflectance Band I1_medium'])
    df["evi"] /= (df['375m Surface Reflectance Band I2_medium'] + 6 * df['375m Surface Reflectance Band I1_medium'] - 7.5 * df['750m Surface Reflectance Band M3_medium'] + 1)
    # Normalized difference tillage index
    df["ndti"] = (df["750m Surface Reflectance Band M10_medium"] - df["750m Surface Reflectance Band M11_medium"]) 
    df["ndti"] /= (df["750m Surface Reflectance Band M10_medium"] + df["750m Surface Reflectance Band M11_medium"]) 
    # Visible Atmospherically Resistant Index
    df["vari_1"] = (df["750m Surface Reflectance Band M4_medium"] - df["375m Surface Reflectance Band I1_medium"]) 
    df["vari_1"] /= (df["750m Surface Reflectance Band M4_medium"] + df["375m Surface Reflectance Band I1_medium"] - df["750m Surface Reflectance Band M3_medium"])
    df["vari_2"] = (df["750m Surface Reflectance Band M4_medium"] - df["750m Surface Reflectance Band M5_medium"])
    df["vari_2"] /= (df["750m Surface Reflectance Band M4_medium"] + df["750m Surface Reflectance Band M5_medium"] - df["750m Surface Reflectance Band M3_medium"])
    # Global Environmental Monitoring Index
    I1 = df["375m Surface Reflectance Band I1_medium"]
    I2 = df["375m Surface Reflectance Band I2_medium"]
    eta = 2*(2*(I2**2 - I1**2) + 1.5 * I2 + 0.5 * I1) / (I2 + I1 + 0.5)
    df["gemi"] = eta * (1-0.25 * eta) - (I1 - 0.125) / (1-I1)
    # Moisture Stress Index
    df["msi_1"] = df["375m Surface Reflectance Band I3_medium"] / df["375m Surface Reflectance Band I2_medium"]
    df["msi_2"] = df["375m Surface Reflectance Band I3_medium"] / df["750m Surface Reflectance Band M7_medium"]
    # Normalized multiband drought index
    I3 = df["375m Surface Reflectance Band I3_medium"]
    M11 = df["750m Surface Reflectance Band M7_medium"]
    df["nmdi"] = (I2 - (I3 - M11)) / (I2 + (I3 - M11))
    
    veg_group = [
        'ndvi', 'ndwi_1', 'ndwi_2', 'ndii_2', 
        'evi', 'ndti', 'vari_1', 'vari_2', 
        'gemi', 'msi_1', 'msi_2', 'nmdi'
    ]
    
    return df, veg_group

In [30]:
# Normalized difference vegetation index
df["ndvi"] = (df["375m Surface Reflectance Band I2_medium"] - df["375m Surface Reflectance Band I1_medium"]) 
df["ndvi"] /= (df["375m Surface Reflectance Band I2_medium"] + df["375m Surface Reflectance Band I1_medium"]) 

In [31]:
# Normalized difference water index
df["ndwi_1"] = (df["375m Surface Reflectance Band I2_medium"] - df["750m Surface Reflectance Band M8_medium"])
df["ndwi_1"] /= (df["375m Surface Reflectance Band I2_medium"] + df["750m Surface Reflectance Band M8_medium"])
df["ndwi_2"] = (df["750m Surface Reflectance Band M7_medium"] - df["750m Surface Reflectance Band M8_medium"]) 
df["ndwi_2"] /= (df["750m Surface Reflectance Band M7_medium"] + df["750m Surface Reflectance Band M8_medium"])

In [32]:
# Normalized difference Infrared index
#df["ndii_1"] = (df["750m Surface Reflectance Band M6_medium"] - df["750m Surface Reflectance Band M10_medium"])
#df["ndii_1"] /= (df["750m Surface Reflectance Band M6_medium"] + df["750m Surface Reflectance Band M10_medium"])
df["ndii_2"] = (df["750m Surface Reflectance Band M7_medium"] - df["750m Surface Reflectance Band M10_medium"])
df["ndii_2"] /= (df["750m Surface Reflectance Band M7_medium"] + df["750m Surface Reflectance Band M10_medium"])

In [33]:
# Enhanced Vegetation Index
df["evi"] = 2.5 * (df['375m Surface Reflectance Band I2_medium'] - df['375m Surface Reflectance Band I1_medium'])
df["evi"] /= (df['375m Surface Reflectance Band I2_medium'] + 6 * df['375m Surface Reflectance Band I1_medium'] - 7.5 * df['750m Surface Reflectance Band M3_medium'] + 1)

In [34]:
# Normalized difference tillage index
df["ndti"] = (df["750m Surface Reflectance Band M10_medium"] - df["750m Surface Reflectance Band M11_medium"]) 
df["ndti"] /= (df["750m Surface Reflectance Band M10_medium"] + df["750m Surface Reflectance Band M11_medium"]) 

In [35]:
# Visible Atmospherically Resistant Index
df["vari_1"] = (df["750m Surface Reflectance Band M4_medium"] - df["375m Surface Reflectance Band I1_medium"]) 
df["vari_1"] /= (df["750m Surface Reflectance Band M4_medium"] + df["375m Surface Reflectance Band I1_medium"] - df["750m Surface Reflectance Band M3_medium"])
df["vari_2"] = (df["750m Surface Reflectance Band M4_medium"] - df["750m Surface Reflectance Band M5_medium"])
df["vari_2"] /= (df["750m Surface Reflectance Band M4_medium"] + df["750m Surface Reflectance Band M5_medium"] - df["750m Surface Reflectance Band M3_medium"])

In [36]:
# Global Environmental Monitoring Index
I1 = df["375m Surface Reflectance Band I1_medium"]
I2 = df["375m Surface Reflectance Band I2_medium"]
eta = 2*(2*(I2**2 - I1**2) + 1.5 * I2 + 0.5 * I1) / (I2 + I1 + 0.5)
df["gemi"] = eta * (1-0.25 * eta) - (I1 - 0.125) / (1-I1)

In [37]:
# Moisture Stress Index
df["msi_1"] = df["375m Surface Reflectance Band I3_medium"] / df["375m Surface Reflectance Band I2_medium"]
df["msi_2"] = df["375m Surface Reflectance Band I3_medium"] / df["750m Surface Reflectance Band M7_medium"]

In [38]:
# Normalized multiband drought index
I3 = df["375m Surface Reflectance Band I3_medium"]
M11 = df["750m Surface Reflectance Band M7_medium"]
df["nmdi"] = (I2 - (I3 - M11)) / (I2 + (I3 - M11))

In [42]:
veg_group = [
    'ndvi', 'ndwi_1', 'ndwi_2', 'ndii_2', 
    'evi', 'ndti', 'vari_1', 'vari_2', 
    'gemi', 'msi_1', 'msi_2', 'nmdi'
]